## Exercise 1 (MNIST Dataset)

* Tensorboard report is not getting converted when pdf is generated, I have added the screenshots of some plots but I have also attached the log files folder which can be used to view the plots and images on a tensorboard running instance.

In [1]:
#pip install tensorboard

In [2]:
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch as pt 
from torch.utils.data import Dataset, DataLoader , Subset, ConcatDataset, ChainDataset
from torch.nn import functional as F
from torch import nn
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter

In [3]:
%load_ext tensorboard

tr_data = MNIST(train = True, root = ".", download = True, transform = ToTensor())
ts_data = MNIST(train = False, root = ".", download = True, transform = ToTensor())


In [4]:
# Here we define our LeNet structure where we have the LeNet structure which is mostly based on this article
# I saw from towardsdatascience https://towardsdatascience.com/implementing-yann-lecuns-lenet-5-in-pytorch-5e05a0911320
# We also define here batch size of 128 and also initialize a summary writer which will help store information about different events.
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5, 5), padding = 2) 
        self.conv2 = nn.Conv2d(6, 16, (5, 5))
        self.fc1   = nn.Linear(16 * 5 * 5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.avg_pool2d(pt.sigmoid(self.conv1(x)), (2, 2))
        x = F.avg_pool2d(pt.sigmoid(self.conv2(x)), (2, 2))
        x = x.view(-1, 16 * 5 * 5)
        x = pt.sigmoid(self.fc1(x))
        x = pt.sigmoid(self.fc2(x))
        x = F.softmax(self.fc3(x), dim = 1)
        return x

b_size = 128
logs = 'runs/experiment_1_mnist'
sum_output = SummaryWriter(logs)

In [5]:
# This function is responisble for training. Since my machine doesnot have gpu so i am working with cpu
# for training which takes some time. Here in for loop I am calculating loss and making prediciton also
# backpropagation is implementing in the same loop. And the result is returned to main function.
def lenet_tr(dataLoad, lenet_model, loss_func, adam_opt):
    dataSize = len(dataLoad.dataset)
    avg_loss, accu = 0, 0
    for batch, (X, y) in tqdm(enumerate(dataLoad), leave = False):
        X, y = X.to('cpu'), y.to('cpu')
        pred = lenet_model(X)
        loss_val = loss_func(pred, y)
        adam_opt.zero_grad()
        loss_val.backward()
        adam_opt.step()
        accu = accu + (pred.argmax(1) == y).type(pt.float).sum().item()
        avg_loss = avg_loss + loss_val.item() * X.size(0)

    accu = accu / dataSize  
    avg_loss = avg_loss / dataSize
    print("Training accuracy: ", str(100 * accu), " Average loss: ", str(avg_loss))
    return avg_loss, (100 * accu)

# Similarly this function is responsible for performing the testing and returning the testing accuracy
# and average loss which is being calculated in the for loop.
def lenet_ts(dataLoad, lenet_model, loss_func):
    dataSize = len(dataLoad.dataset)
    ts_loss, accu = 0, 0

    with pt.no_grad():
        for X, y in tqdm(dataLoad, leave = False):
            X, y = X.to('cpu'), y.to('cpu')
            pred = lenet_model(X)
            ts_loss = ts_loss + loss_func(pred, y).item()*X.size(0)
            accu = accu + (pred.argmax(1) == y).type(pt.float).sum().item()

    ts_loss = ts_loss / dataSize
    accu = accu / dataSize
    print("Testing accuracy: ", str(100 * accu), " Average loss: ", str(ts_loss))
    return ts_loss, (100 * accu)


In [6]:
# This is the main part of the code where we are using first dataloader to get our training and testing data
# ready for the model and then training our model for different learning rates as mentioned in the assignment,
# Here i have focused on only Adam optimizer with different learning rates. And i am using summaryWriter to
# store important data which could be useful to see later on such as train/test accuracy/loss with different
# learning rates.
for lr in [0.1, 0.01, 0.001]:
    tr_dl = DataLoader(tr_data, batch_size = b_size, shuffle = True)
    ts_dl = DataLoader(ts_data, batch_size = b_size, shuffle = False)

    lenet_model = LeNet()
    lenet_model.to('cpu')
    loss_func = nn.CrossEntropyLoss()

    adam_opt = pt.optim.Adam(lenet_model.parameters(), lr = lr, betas=(0.9, 0.999), eps = 1e-08, weight_decay = 0, amsgrad = False)

    for i in tqdm(range(30)):
        print("Epoch: ", i + 1)
        tr_loss, tr_accu = lenet_tr(tr_dl, lenet_model, loss_func, adam_opt)
        ts_loss, ts_accu = lenet_ts(ts_dl, lenet_model, loss_func)    
        if sum_output is not None:
            sum_output.add_scalars(f'Loss with lr = {lr}', {"Train" : tr_loss,
                                        "Test":ts_loss}, i)
            sum_output.add_scalars(f'Accuracy with lr = {lr}', {"Train" : tr_accu,
                                            "Test" : ts_accu}, i)
            
            sum_output.add_scalars(f'Train_Accuracy with lr = ', {f"{lr}" : tr_accu}, i)
            sum_output.add_scalars(f'Test_Accuracy with lr = ', {f"{lr}" : ts_accu}, i)

            sum_output.add_scalars(f'Train_loss with lr = ', {f"{lr}" : tr_loss}, i)
            sum_output.add_scalars(f'Test_loss with lr = ', {f"{lr}" : ts_loss}, i)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:  1


0it [00:00, ?it/s]

Training accuracy:  11.185  Average loss:  2.3476763763427733


  0%|          | 0/79 [00:00<?, ?it/s]

...

Testing accuracy:  98.35000000000001  Average loss:  1.4783416591644287
Epoch:  30


0it [00:00, ?it/s]

Training accuracy:  98.69333333333333  Average loss:  1.475559120241801


  0%|          | 0/79 [00:00<?, ?it/s]

Testing accuracy:  98.33  Average loss:  1.4781860063552856


In [7]:
# Here we are making an activation sample first and then using hooks on our conv layers just to get the
# state of each layer and later on saving their output using our summarywrite 
a_sample = iter(tr_dl).next()[0][np.random.randint(b_size)]

activList = dict()
def get_activation(name):
    def hook(model, input, output):
        activList[name] = output.detach()
    return hook

lenet_model.conv1.register_forward_hook(get_activation('conv1'))
lenet_model.conv2.register_forward_hook(get_activation('conv2'))
lenet_model.cpu()

sum_output.add_image(f'Original Image', a_sample)

output = lenet_model(a_sample.unsqueeze(0))

active_1st = activList['conv1'].squeeze()
active_2nd = activList['conv2'].squeeze()

filtrs = active_1st.size(0)
fig, axes = plt.subplots(1, filtrs, figsize = (filtrs * 2, 2))
for i in range(filtrs):
    axes[i].imshow(active_1st[i].numpy())
    axes[i].axis('off')

sum_output.add_figure('1st Conv. layer', fig)
plt.close()

filtrs = active_2nd.size(0)
fig, axes = plt.subplots(4, filtrs // 4, figsize = (filtrs // 2, 8))
for i in range(filtrs):
    x, y = i%4,i//4 
    axes[x, y].imshow(active_2nd[i].numpy())
    axes[x, y].axis('off')

sum_output.add_figure('2nd Conv. layer', fig)
plt.close()

listKernel = lenet_model.conv1.weight.detach()
filtrs = listKernel.size(0)
fig, axes = plt.subplots(1, filtrs, figsize = (filtrs * 2, 2))
for idx in range(filtrs):
    axes[idx].imshow(listKernel[idx].squeeze())
    axes[idx].axis('off')

sum_output.add_figure('1st Conv. feature-map', fig)
plt.close()

listKernel = lenet_model.conv2.weight.detach()
fig, axes = plt.subplots(listKernel.size(1), listKernel.size(0), figsize = (listKernel.size(0), listKernel.size(1)))
for i in range(listKernel.size(0)):
    for j in range(listKernel.size(1)):
        axes[j, i].imshow(listKernel[i, j].squeeze())
        axes[j, i].axis('off')

sum_output.add_figure('2nd Conv. feature-map', fig)
plt.close()
sum_output.close()

In [10]:
# To run the tensorbord in jupyter notebook and see the diagonistic data from our summarywriter.
%tensorboard --logdir runs/experiment_1_mnist --host localhost --port 8095
#